In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
d1='petrol cars are cheaper than diesel cars'

d2='diesel is cheaper than petrol'

d3='diesel cars are costlier than petrol cars'

d4= 'petrol is costlier than diesel'

In [3]:
# create our dataframe
d=[['d1',d1],['d2',d2],['d3',d3],['d4',d4]]
df=pd.DataFrame(d,columns=['d','ds'])

In [4]:
df

,d,ds
0,d1,petrol cars are cheaper than diesel cars
1,d2,diesel is cheaper than petrol
2,d3,diesel cars are costlier than petrol cars
3,d4,petrol is costlier than diesel


In [5]:
vectorizer = CountVectorizer(stop_words='english')
embeddings = vectorizer.fit_transform(df['ds'])


# create our dataframe for countvectorizer
cvdf = pd.DataFrame(embeddings.toarray(),columns=vectorizer.get_feature_names_out())
cvdf

,cars,cheaper,costlier,diesel,petrol
0,2,1,0,1,1
1,0,1,0,1,1
2,2,0,1,1,1
3,0,0,1,1,1


In [6]:
#Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
tfidf = TfidfVectorizer(stop_words='english')

In [7]:
#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(df['ds'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape

(4, 5)

In [8]:
print(tfidf_matrix)

  (0, 3)	0.2730470684276293
  (0, 1)	0.41252650747464825
  (0, 0)	0.8250530149492965
  (0, 4)	0.2730470684276293
  (1, 3)	0.4832218287328064
  (1, 1)	0.730063920812585
  (1, 4)	0.4832218287328064
  (2, 2)	0.41252650747464825
  (2, 3)	0.2730470684276293
  (2, 0)	0.8250530149492965
  (2, 4)	0.2730470684276293
  (3, 2)	0.730063920812585
  (3, 3)	0.4832218287328064
  (3, 4)	0.4832218287328064


In [9]:
print("Feature Names n",tfidf.get_feature_names_out())

Feature Names n ['cars' 'cheaper' 'costlier' 'diesel' 'petrol']


In [10]:
for i, feature in enumerate(tfidf.get_feature_names_out()):
    print(i, feature)

0 cars
1 cheaper
2 costlier
3 diesel
4 petrol


In [11]:
df_tfidf_sklearn = pd.DataFrame(tfidf_matrix.toarray(),columns=tfidf.get_feature_names_out())
df_tfidf_sklearn

,cars,cheaper,costlier,diesel,petrol
0,0.825053,0.412527,0.000000,0.273047,0.273047
1,0.000000,0.730064,0.000000,0.483222,0.483222
2,0.825053,0.000000,0.412527,0.273047,0.273047
3,0.000000,0.000000,0.730064,0.483222,0.483222


In [12]:
X=tfidf_matrix.toarray()
Y=tfidf_matrix.toarray()

In [13]:
X@Y.T

array([[1.        , 0.56505533, 0.82982188, 0.26388461],
       [0.56505533, 1.        , 0.26388461, 0.46700667],
       [0.82982188, 0.26388461, 1.        , 0.56505533],
       [0.26388461, 0.46700667, 0.56505533, 1.        ]])

In [14]:
# Import linear_kernel
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [15]:
cosine_sim.shape

(4, 4)

In [16]:
cosine_sim

array([[1.        , 0.56505533, 0.82982188, 0.26388461],
       [0.56505533, 1.        , 0.26388461, 0.46700667],
       [0.82982188, 0.26388461, 1.        , 0.56505533],
       [0.26388461, 0.46700667, 0.56505533, 1.        ]])

In [17]:
cos=pd.DataFrame(cosine_sim,index=['d1','d2','d3','d4'],columns=['d1','d2','d3','d4'])
cos

,d1,d2,d3,d4
d1,1.000000,0.565055,0.829822,0.263885
d2,0.565055,1.000000,0.263885,0.467007
d3,0.829822,0.263885,1.000000,0.565055
d4,0.263885,0.467007,0.565055,1.000000


In [18]:
#Construct a reverse map of indices and document names
indices = pd.Series(df.index, index=df['d']).drop_duplicates()


In [19]:
indices


d
d1    0
d2    1
d3    2
d4    3
dtype: int64

In [20]:
indices['d3']

2

In [21]:
sim_scor=list(enumerate(cosine_sim[1]))
sim_scor

[(0, 0.5650553269575254),
 (1, 0.9999999999999996),
 (2, 0.26388460747146153),
 (3, 0.4670066715277553)]

In [22]:
sim_scor=sorted(sim_scor, key=lambda x: x[1], reverse=True)
sim_scor

[(1, 0.9999999999999996),
 (0, 0.5650553269575254),
 (3, 0.4670066715277553),
 (2, 0.26388460747146153)]

In [23]:
# Get the scores of the 3 most similar documents
sim_scor = sim_scor[1:4]
sim_scor

[(0, 0.5650553269575254), (3, 0.4670066715277553), (2, 0.26388460747146153)]

In [24]:
# Get the indices
sim_scor[0][0]

0

In [25]:
ind=[i[0] for i in sim_scor]

In [26]:
ind

[0, 3, 2]

In [27]:
df.d.iloc[ind]

0    d1
3    d4
2    d3
Name: d, dtype: object

In [28]:
pd.DataFrame(df[['d','ds']].iloc[ind])

,d,ds
0,d1,petrol cars are cheaper than diesel cars
3,d4,petrol is costlier than diesel
2,d3,diesel cars are costlier than petrol cars
